# DEA Coastlines generation using command line tools <img align="right" src="https://github.com/GeoscienceAustralia/dea-notebooks/raw/develop/Supplementary_data/dea_logo.jpg">
This notebook demonstrates how to run a DEA Coastlines analysis using command line tools.

### Setup

Set working directory to top level of repo to ensure links work correctly:

In [1]:
cd ..

/home/jovyan/Robbi/dea-coastlines


Update required packages:

In [2]:
pip install -r requirements.in --quiet

You should consider upgrading via the '/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Set analysis parameters:

In [11]:
config_path = 'configs/dea_coastlines_config.yaml'
study_area = 190
raster_version = 'testing'
vector_version = 'testing'
continental_version = 'testing'

### Run DEA Coastlines analysis
#### Run tidally-constrained raster generation

In [4]:
!python -m coastlines.raster --help

Usage: python -m coastlines.raster [OPTIONS]

Options:
  --config_path TEXT              Path to the YAML config file defining inputs
                                  to use for this analysis. These are
                                  typically located in the `dea-
                                  coastlines/configs/` directory.  [required]
  --study_area TEXT               A string providing a unique ID of an
                                  analysis gridcell that will be used to run
                                  the analysis. This should match a row in the
                                  "id" column of the provided analysis
                                  gridcell vector file.  [required]
  --raster_version TEXT           A unique string proving a name that will be
                                  used for output raster directories and
                                  files. This can be used to version different
                                  analysis outputs.  [req

Example analysis:

In [5]:
!python -m coastlines.raster --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --start_year 1988 --end_year 2021

<Client: 'tcp://127.0.0.1:40407' processes=1 threads=15, memory=117.21 GiB>
2022-11-16 02:11:13 INFO Study area 329: Loaded study area grid
2022-11-16 02:11:17 INFO Study area 329: Loaded virtual product
Creating reduced resolution tide modelling array
Modelling tides using FES2014 tide model
Reprojecting tides into original array
100%|█████████████████████████████████████████| 434/434 [00:07<00:00, 58.99it/s]
2022-11-16 02:12:03 INFO Study area 329: Finished modelling tide heights
2022-11-16 02:12:03 INFO Study area 329: Calculating low and high tide cutoffs for each pixel
2022-11-16 02:12:03 INFO Study area 329: Started exporting raster data
/env/lib/python3.8/site-packages/dask/core.py:119: RuntimeWarning: divide by zero encountered in true_divide
  return func(*(_execute_task(a, cache) for a in args))
2022-11-16 02:14:10 INFO Study area 329: Completed exporting raster data


#### Run vector annual shoreline and rates of change statistics generation

In [6]:
!python -m coastlines.vector --help

Usage: python -m coastlines.vector [OPTIONS]

Options:
  --config_path TEXT            Path to the YAML config file defining inputs
                                to use for this analysis. These are typically
                                located in the `dea-coastlines/configs/`
                                directory.  [required]
  --study_area TEXT             A string providing a unique ID of an analysis
                                gridcell that was previously used to generate
                                raster files. This is used to identify the
                                raster files that will be used as inputs for
                                shoreline extraction, and should match a row
                                in the "id" column of the provided analysis
                                gridcell vector file.  [required]
  --raster_version TEXT         A unique string providing a name that was used
                                to generate raster files

Example analysis:

In [10]:
!python -m coastlines.vector --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --vector_version {vector_version} --start_year 1988 --end_year 2021 --baseline_year 2021

2022-11-30 01:17:02 INFO Study area 6: Starting vector generation
2022-11-30 01:17:09 INFO Study area 6: Loaded rasters
2022-11-30 01:17:10 INFO Study area 6: Loaded ocean points
Operating in single z-value, multiple arrays mode
Failed to generate any valid contours; verify that values passed to `z_values` are valid and present in `da`
2022-11-30 01:17:39 ERROR Study area 6: Failed to run process with error Study area 6: Unable to extract any valid shorelines
Traceback (most recent call last):
  File "/home/jovyan/Robbi/dea-coastlines/coastlines/vector.py", line 1803, in generate_vectors_cli
    generate_vectors(
  File "/home/jovyan/Robbi/dea-coastlines/coastlines/vector.py", line 1457, in generate_vectors
    raise ValueError(
ValueError: Study area 6: Unable to extract any valid shorelines


#### Run continental-scale layer generation

In [8]:
!python -m coastlines.continental --help

Usage: python -m coastlines.continental [OPTIONS]

Options:
  --vector_version TEXT           A unique string proving a name that was used
                                  for output vector directories and files.
                                  This is used to identify the tiled annual
                                  shoreline and rates of change layers that
                                  will be combined into continental-scale
                                  layers.  [required]
  --continental_version TEXT      A unique string proving a name that will be
                                  used for output continental-scale layers.
                                  This allows multiple versions of
                                  continental-scale layers to be generated
                                  from the same input vector data, e.g. for
                                  testing different hotspot of coastal change
                                  summary layers. If not

Example analysis:

In [9]:
!python -m coastlines.continental --vector_version {vector_version} --continental_version {continental_version} --shorelines True --ratesofchange True --hotspots True --baseline_year 2021

2022-11-16 02:14:59 INFO Writing data to data/processed/testing
Warning 1: A geometry of type MULTILINESTRING is inserted into layer shorelines_annual of geometry type LINESTRING, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
2022-11-16 02:15:01 INFO Merging annual shorelines complete
2022-11-16 02:15:03 INFO Merging rates of change points complete
2022-11-16 02:15:03 INFO Generating continental hotspots
2022-11-16 02:15:05 INFO Generating continental hotspots
2022-11-16 02:15:05 INFO Calculating 10000 m hotspots
2022-11-16 02:15:06 INFO Calculating 5000 m hotspots
2022-11-16 02:15:07 INFO Calculating 1000 m hotspots
2022-11-16 02:15:12 INFO Writing hotspots complete
2022-11-16 02:15:32 INFO Writing rates of change points to

## Example combined analysis
This demonstrates how the three components of DEA Coastlines (raster generation, vector generation and continental layers generation) can be applied automatically to a sequence of input study area grid cells.

In [5]:
# Study areas
study_areas = [329]

In [6]:
# Run raster and vector generation for each study area
for study_area in study_areas:
    print(study_area)
    !python -m coastlines.raster --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --start_year 1988 --end_year 2021
    !python -m coastlines.vector --config_path {config_path} --study_area {study_area} --raster_version {raster_version} --vector_version {vector_version} --start_year 1988 --end_year 2021 --baseline_year 2021
    
# When complete, combine into single continental outputs
!python -m coastlines.continental --vector_version {vector_version} --continental_version {continental_version} --shorelines True --ratesofchange True --hotspots True --baseline_year 2021

586
2022-08-17 05:05:29 INFO Study area 586: Starting vector generation
2022-08-17 05:05:54 INFO Study area 586: Loaded rasters
2022-08-17 05:05:55 INFO Study area 586: Loaded tide modelling points
2022-08-17 05:06:37 INFO Study area 586: Extracted annual shorelines
2022-08-17 05:06:37 INFO Study area 586: Extracted rates of change points
2022-08-17 05:07:12 INFO Study area 586: Calculated distances to each annual shoreline
2022-08-17 05:07:15 INFO Study area 586: Calculated rates of change regressions
2022-08-17 05:07:21 INFO Study area 586: Calculated all of time statistics
2022-08-17 05:07:21 INFO Study area 586: Calculated rate of change certainty flags
2022-08-17 05:07:21 INFO Study area 586: Added region attributes and geohash UIDs
2022-08-17 05:07:27 INFO Study area 586: Output vector files written to data/interim/vector/c3_grid/586_c3_grid
2022-08-17 05:07:30 INFO Writing data to data/processed/c3_grid
Warning 1: A geometry of type MULTILINESTRING is inserted into layer shoreli

### Clean up and remove files
Warning: this will delete all data processed with a matching unique version name. Only run these cells if you no longer need these processed outputs!

In [ ]:
# rm -rf data/interim/raster/{raster_version}

In [ ]:
# rm -rf data/interim/vector/{vector_version}

In [ ]:
# rm -rf data/processed/{continental_version}

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** For assistance with any of the Python code or Jupyter Notebooks in this repository, please post a [Github issue](https://github.com/GeoscienceAustralia/dea-coastlines/issues/new).

**Last modified:** November 2022

**To cite:**

> Bishop-Taylor, R., Nanson, R., Sagar, S., Lymburner, L. (2021). Mapping Australia's dynamic coastline at mean sea level using three decades of Landsat imagery. Remote Sensing of Environment, 267, 112734. Available: https://doi.org/10.1016/j.rse.2021.112734
>
> Nanson, R., Bishop-Taylor, R., Sagar, S., Lymburner, L., (2022). Geomorphic insights into Australia's coastal change using a national dataset derived from the multi-decadal Landsat archive. Estuarine, Coastal and Shelf Science, 265, p.107712. Available: https://doi.org/10.1016/j.ecss.2021.107712
>
> Bishop-Taylor, R., Sagar, S., Lymburner, L., Alam, I., Sixsmith, J. (2019). Sub-pixel waterline extraction: characterising accuracy and sensitivity to indices and spectra. Remote Sensing, 11 (24):2984. Available: https://doi.org/10.3390/rs11242984